In [1]:
# !wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
# !unzip uncased_L-12_H-768_A-12.zip
# !wget https://raw.githubusercontent.com/huseinzol05/NLP-Models-Tensorflow/master/abstractive-summarization/dataset/ctexts.json
# !wget https://raw.githubusercontent.com/huseinzol05/NLP-Models-Tensorflow/master/abstractive-summarization/dataset/headlines.json

In [2]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling
import numpy as np
import tensorflow as tf
from tensor2tensor.utils import beam_search

/usr/local/lib/python3.6/dist-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)
/usr/local/lib/python3.6/dist-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)
/usr/local/lib/python3.6/dist-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)
/usr/local/lib/python3.6/dist-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)
/usr/local/lib/python3.6/dist-pa

In [3]:
import json
from unidecode import unidecode
import re

In [4]:
with open('ctexts.json','r') as fopen:
    ctexts = json.load(fopen)
    
with open('headlines.json','r') as fopen:
    headlines = json.load(fopen)

In [5]:
BERT_VOCAB = 'uncased_L-12_H-768_A-12/vocab.txt'
BERT_INIT_CHKPNT = 'uncased_L-12_H-768_A-12/bert_model.ckpt'
BERT_CONFIG = 'uncased_L-12_H-768_A-12/bert_config.json'

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

def topic_modelling(string, n = 200):
    vectorizer = TfidfVectorizer()
    tf = vectorizer.fit_transform([string])
    tf_features = vectorizer.get_feature_names()
    compose = TruncatedSVD(1).fit(tf)
    return ' '.join([tf_features[i] for i in compose.components_[0].argsort()[: -n - 1 : -1]])

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary inc

In [7]:
import warnings
warnings.simplefilter("ignore", category=PendingDeprecationWarning)
warnings.simplefilter("ignore", category=RuntimeWarning)

In [8]:
%%time
h, c = [], []
for i in range(len(ctexts)):
    try:
        c.append(topic_modelling(ctexts[i]))
        h.append(headlines[i])
    except:
        pass

CPU times: user 37.5 s, sys: 51.9 s, total: 1min 29s
Wall time: 22.5 s


In [9]:
import collections
import itertools

def build_dataset(words, n_words, atleast=1):
    count = [['PAD', 0], ['GO', 1], ['EOS', 2], ['UNK', 3]]
    counter = collections.Counter(words).most_common(n_words)
    counter = [i for i in counter if i[1] >= atleast]
    count.extend(counter)
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [10]:
concat = ' '.join(h).split()
vocabulary_size = len(list(set(concat)))
data, count, dictionary, rev_dictionary = build_dataset(concat, vocabulary_size)
print('vocab from size: %d'%(vocabulary_size))
print('Most common words', count[4:10])
print('Sample data', data[:10], [rev_dictionary[i] for i in data[:10]])
print('filtered vocab size:',len(dictionary))
print("% of vocab used: {}%".format(round(len(dictionary)/vocabulary_size,4)*100))

vocab from size: 8534
Most common words [('to', 1388), ('in', 1196), ('comma', 876), ('s', 785), ('for', 733), ('of', 596)]
Sample data [2797, 14, 2798, 2799, 656, 2800, 5, 1642, 657, 2086] ['daman', 'and', 'diu', 'revokes', 'mandatory', 'rakshabandhan', 'in', 'offices', 'order', 'malaika']
filtered vocab size: 8538
% of vocab used: 100.05%


In [11]:
MAX_SEQ_LENGTH = 200
tokenization.validate_case_matches_checkpoint(True,BERT_INIT_CHKPNT)
tokenizer = tokenization.FullTokenizer(
      vocab_file=BERT_VOCAB, do_lower_case=True)

In [12]:
from tqdm import tqdm

input_ids, input_masks, segment_ids = [], [], []

for text in tqdm(c):
    tokens_a = tokenizer.tokenize(text)
    if len(tokens_a) > MAX_SEQ_LENGTH - 2:
        tokens_a = tokens_a[:(MAX_SEQ_LENGTH - 2)]
    tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
    segment_id = [0] * len(tokens)
    input_id = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_id)
    padding = [0] * (MAX_SEQ_LENGTH - len(input_id))
    input_id += padding
    input_mask += padding
    segment_id += padding
    
    input_ids.append(input_id)
    input_masks.append(input_mask)
    segment_ids.append(segment_id)

100%|██████████| 4395/4395 [00:14<00:00, 295.96it/s]


In [13]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)

In [14]:
GO = dictionary['GO']
PAD = dictionary['PAD']
EOS = dictionary['EOS']
UNK = dictionary['UNK']

In [15]:
for i in range(len(h)):
    h[i] = h[i] + ' EOS'
h[0]

'daman and diu revokes mandatory rakshabandhan in offices order EOS'

In [16]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)
epoch = 20
batch_size = 5
warmup_proportion = 0.1
num_train_steps = int(len(input_ids) / batch_size * epoch)
num_warmup_steps = int(num_train_steps * warmup_proportion)

In [17]:
def embed_seq(x, vocab_sz, embed_dim, name, zero_pad=True): 
    embedding = tf.get_variable(name, [vocab_sz, embed_dim]) 
    if zero_pad:
        embedding = tf.concat([tf.zeros([1, embed_dim]), embedding[1:, :]], 0) 
    x = tf.nn.embedding_lookup(embedding, x)
    return x

def position_encoding(inputs):
    T = tf.shape(inputs)[1]
    repr_dim = inputs.get_shape()[-1].value
    pos = tf.reshape(tf.range(0.0, tf.to_float(T), dtype=tf.float32), [-1, 1])
    i = np.arange(0, repr_dim, 2, np.float32)
    denom = np.reshape(np.power(10000.0, i / repr_dim), [1, -1])
    enc = tf.expand_dims(tf.concat([tf.sin(pos / denom), tf.cos(pos / denom)], 1), 0)
    return tf.tile(enc, [tf.shape(inputs)[0], 1, 1])

def layer_norm(inputs, epsilon=1e-8):
    mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)
    normalized = (inputs - mean) / (tf.sqrt(variance + epsilon))
    params_shape = inputs.get_shape()[-1:]
    gamma = tf.get_variable('gamma', params_shape, tf.float32, tf.ones_initializer())
    beta = tf.get_variable('beta', params_shape, tf.float32, tf.zeros_initializer())
    return gamma * normalized + beta


def cnn_block(x, dilation_rate, pad_sz, hidden_dim, kernel_size):
    x = layer_norm(x)
    pad = tf.zeros([tf.shape(x)[0], pad_sz, hidden_dim])
    x =  tf.layers.conv1d(inputs = tf.concat([pad, x, pad], 1),
                          filters = hidden_dim,
                          kernel_size = kernel_size,
                          dilation_rate = dilation_rate)
    x = x[:, :-pad_sz, :]
    x = tf.nn.relu(x)
    return x

class Translator:
    def __init__(self, size_layer, num_layers, embedded_size, 
                 to_dict_size, learning_rate, kernel_size = 2, n_attn_heads = 16):
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype=tf.int32)
        batch_size = tf.shape(self.X)[0]
        self.batch_size = batch_size
        
        main = tf.strided_slice(self.Y, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        self.embedding = tf.Variable(tf.random_uniform([to_dict_size, embedded_size], -1, 1))
        
        self.num_layers = num_layers
        self.kernel_size = kernel_size
        self.size_layer = size_layer
        self.n_attn_heads = n_attn_heads
        self.dict_size = to_dict_size
        
        self.training_logits = self.forward(self.X, decoder_input)

        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.training_logits,
                                                     targets = self.Y,
                                                     weights = masks)
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        y_t = tf.argmax(self.training_logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
    def forward(self, x, y, reuse = False):
        with tf.variable_scope('forward',reuse=reuse):
            with tf.variable_scope('forward',reuse=reuse):
                model = modeling.BertModel(
                config=bert_config,
                is_training=True,
                input_ids=x,
                use_one_hot_embeddings=False)
                encoder_embedded = model.get_sequence_output()
                decoder_embedded = tf.nn.embedding_lookup(self.embedding, y)
                
                g = tf.identity(decoder_embedded)
                for i in range(self.num_layers):
                    dilation_rate = 2 ** i
                    pad_sz = (self.kernel_size - 1) * dilation_rate
                    with tf.variable_scope('decode_%d'%i,reuse=reuse):
                        attn_res = h = cnn_block(decoder_embedded, dilation_rate, 
                                                 pad_sz, self.size_layer, self.kernel_size)
                        C = []
                        for j in range(self.n_attn_heads):
                            h_ = tf.layers.dense(h, self.size_layer//self.n_attn_heads)
                            g_ = tf.layers.dense(g, self.size_layer//self.n_attn_heads)
                            zu_ = tf.layers.dense(encoder_embedded, self.size_layer//self.n_attn_heads)
                            ze_ = tf.layers.dense(encoder_embedded, self.size_layer//self.n_attn_heads)

                            d = tf.layers.dense(h_, self.size_layer//self.n_attn_heads) + g_
                            dz = tf.matmul(d, tf.transpose(zu_, [0, 2, 1]))
                            a = tf.nn.softmax(dz)
                            c_ = tf.matmul(a, ze_)
                            C.append(c_)

                        c = tf.concat(C, 2)
                        h = tf.layers.dense(attn_res + c, self.size_layer)
                        decoder_embedded += h

                return tf.layers.dense(decoder_embedded, self.dict_size)

In [18]:
size_layer = 256
num_layers = 4
embedded_size = 256
learning_rate = 2e-5

In [19]:
def beam_search_decoding(length = 20, beam_width = 5):
    initial_ids = tf.fill([model.batch_size], GO)
    
    def symbols_to_logits(ids):
        x = tf.contrib.seq2seq.tile_batch(model.X, beam_width)
        logits = model.forward(x, ids, reuse = True)
        return logits[:, tf.shape(ids)[1]-1, :]

    final_ids, final_probs, _ = beam_search.beam_search(
        symbols_to_logits,
        initial_ids,
        beam_width,
        length,
        len(dictionary),
        0.0,
        eos_id = EOS)
    
    return final_ids

In [20]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Translator(size_layer, num_layers, embedded_size, 
                len(dictionary), learning_rate)
model.generate = beam_search_decoding()
sess.run(tf.global_variables_initializer())

/usr/local/lib/python3.6/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


In [21]:
def get_assignment_map_from_checkpoint(tvars, init_checkpoint):
    """Compute the union of the current variables and checkpoint variables."""
    assignment_map = {}
    initialized_variable_names = {}

    name_to_variable = collections.OrderedDict()
    for var in tvars:
        name = var.name
        m = re.match('^(.*):\\d+$', name)
        if m is not None:
            name = m.group(1)
        name_to_variable[name] = var

    init_vars = tf.train.list_variables(init_checkpoint)

    assignment_map = collections.OrderedDict()
    for x in init_vars:
        (name, var) = (x[0], x[1])
        if 'bert' not in name:
            continue
        assignment_map[name] = name_to_variable['forward/forward/'+name]
        initialized_variable_names[name] = 1
        initialized_variable_names[name + ':0'] = 1

    return (assignment_map, initialized_variable_names)

In [22]:
tvars = tf.trainable_variables()
assignment_map, initialized_variable_names = get_assignment_map_from_checkpoint(tvars, 
                                                                                BERT_INIT_CHKPNT)

In [23]:
saver = tf.train.Saver(var_list = assignment_map)
saver.restore(sess, BERT_INIT_CHKPNT)

INFO:tensorflow:Restoring parameters from uncased_L-12_H-768_A-12/bert_model.ckpt


In [24]:
def str_idx(corpus, dic):
    X = []
    for i in corpus:
        ints = []
        for k in i.split():
            ints.append(dic.get(k,UNK))
        X.append(ints)
    return X

In [25]:
Y = str_idx(h, dictionary)

In [26]:
from sklearn.cross_validation import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(input_ids, Y, test_size = 0.05)

/home/husein/.local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [27]:
def pad_sentence_batch(sentence_batch, pad_int):
    padded_seqs = []
    seq_lens = []
    max_sentence_len = max([len(sentence) for sentence in sentence_batch])
    for sentence in sentence_batch:
        padded_seqs.append(sentence + [pad_int] * (max_sentence_len - len(sentence)))
        seq_lens.append(len(sentence))
    return padded_seqs, seq_lens

In [28]:
sess.run(model.generate, feed_dict = {model.X: [test_X[0]]})

array([[[   1, 4700, 1519, 4183, 1519, 6135, 1519,  725,  982, 1045,
         7026, 8370, 1519,   42, 2505, 5440,  567, 2025, 6334, 7626,
         6152],
        [   1, 4700, 1519, 4183, 1519, 6135, 1519,  725,  982, 1045,
         7026, 8370, 1519,   42, 2505, 5440,  567, 2025, 6334, 4224,
          932],
        [   1, 4700, 1519, 4183, 1519, 6135, 1519,  725,  982, 1045,
         7026, 8370, 1519,   42, 2505, 5440,  567, 5651, 1519, 5578,
         5423],
        [   1, 4700, 1519, 4183, 1519, 6135, 1519,  725,  982, 1045,
         7026, 8370, 1519,   42, 2505, 5440,  567, 5651, 1519, 5578,
          434],
        [   1, 4700, 1519, 4183, 1519, 6135, 1519,  725,  982, 1045,
         7026, 8370, 1519,   42, 2505, 5440,  567, 5651, 1519, 8244,
         3935]]], dtype=int32)

In [29]:
import time

for EPOCH in range(10):
    lasttime = time.time()

    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    pbar = tqdm(
        range(0, len(train_X), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_X))
        batch_x = train_X[i : index]
        batch_y, _ = pad_sentence_batch(train_Y[i : index], PAD)
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x
            },
        )
        assert not np.isnan(cost)
        train_loss += cost
        train_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_X), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_X))
        batch_x = test_X[i : index]
        batch_y, _ = pad_sentence_batch(test_Y[i : index], PAD)
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x
            },
        )
        test_loss += cost
        test_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    train_loss /= len(train_X) / batch_size
    train_acc /= len(train_X) / batch_size
    test_loss /= len(test_X) / batch_size
    test_acc /= len(test_X) / batch_size
        
    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )

train minibatch loop:   0%|          | 0/835 [00:00<?, ?it/s]

time taken: 600.4188756942749
epoch: 0, training loss: 7.504465, training acc: 0.103313, valid loss: 7.265235, valid acc: 0.113662



train minibatch loop:   0%|          | 0/835 [00:00<?, ?it/s]

time taken: 595.078351020813
epoch: 1, training loss: 6.944261, training acc: 0.116727, valid loss: 7.155360, valid acc: 0.119874



train minibatch loop:   0%|          | 0/835 [00:00<?, ?it/s]

...

time taken: 595.3370399475098
epoch: 7, training loss: 5.039291, training acc: 0.200931, valid loss: 6.947434, valid acc: 0.142766



train minibatch loop:   0%|          | 0/835 [00:00<?, ?it/s]

time taken: 594.4455664157867
epoch: 8, training loss: 4.733280, training acc: 0.219349, valid loss: 7.004194, valid acc: 0.138716



test minibatch loop: 100%|██████████| 44/44 [00:11<00:00,  3.80it/s, accuracy=0.0769, cost=7.38]

time taken: 594.7499196529388
epoch: 9, training loss: 4.430527, training acc: 0.243508, valid loss: 7.084307, valid acc: 0.135056



In [30]:
generated = [rev_dictionary[i] for i in sess.run(model.generate, feed_dict = {model.X: [test_X[0]]})[0,0,:]]
' '.join(generated)

'GO govt bans to set in maharashtra EOS PAD'

In [31]:
' '.join([rev_dictionary[i] for i in test_Y[0]])

'1l for pregnant women to expose sex determination centres EOS'